# HTMLHeaderTextSplitter Tutorial

## 개요

`HTMLHeaderTextSplitter`는 HTML 문서의 구조를 인식하여 텍스트를 분할하는 LangChain의 텍스트 분할기입니다.

### 주요 특징
- HTML 헤더 태그(h1, h2, h3 등)를 기준으로 텍스트 분할
- 각 청크에 헤더 계층 구조를 메타데이터로 보존
- 문서의 의미론적 구조 유지
- 다른 텍스트 분할기와 파이프라인 구성 가능

### 사용 사례
- 웹 스크래핑된 HTML 문서 처리
- 구조화된 문서의 계층적 정보 보존
- RAG 시스템에서 문맥 정보가 중요한 경우

## 1. HTML 문자열 분할

### 기본 사용법
- `headers_to_split_on`: 분할 기준이 되는 헤더 태그와 메타데이터 이름을 튜플로 지정
- 각 헤더 레벨의 텍스트가 해당 헤더의 메타데이터와 함께 분할됨

In [1]:
from langchain_text_splitters import HTMLHeaderTextSplitter

# 테스트용 HTML 문자열
html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>헤더1</h1>
        <p>헤더1 에 포함된 본문</p>
        <div>
            <h2>헤더2-1 제목</h2>
            <p>헤더2-1 에 포함된 본문</p>
            <h3>헤더3-1 제목</h3>
            <p>헤더3-1 에 포함된 본문</p>
            <h3>헤더3-2 제목</h3>
            <p>헤더3-2 에 포함된 본문</p>
        </div>
        <div>
            <h2>헤더2-2 제목2</h2>
            <p>헤더2-2 에 포함된 본문</p>
        </div>
        <br>
        <p>마지막 내용</p>
    </div>
</body>
</html>
"""

# 분할 기준 헤더 설정
headers_to_split_on = [
    ("h1", "Header 1"),  # (태그명, 메타데이터 키)
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

# HTMLHeaderTextSplitter 생성
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# HTML 분할 실행
html_header_splits = html_splitter.split_text(html_string)

# 결과 출력
for header in html_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

헤더1
{}
헤더1 에 포함된 본문  
헤더2-1 제목 헤더3-1 제목 헤더3-2 제목
{'Header 1': '헤더1'}
헤더2-1 에 포함된 본문
{'Header 1': '헤더1', 'Header 2': '헤더2-1 제목'}
헤더3-1 에 포함된 본문
{'Header 1': '헤더1', 'Header 2': '헤더2-1 제목', 'Header 3': '헤더3-1 제목'}
헤더3-2 에 포함된 본문
{'Header 1': '헤더1', 'Header 2': '헤더2-1 제목', 'Header 3': '헤더3-2 제목'}
헤더2-2 제목2
{'Header 1': '헤더1'}
헤더2-2 에 포함된 본문
{'Header 1': '헤더1', 'Header 2': '헤더2-2 제목2'}
마지막 내용
{'Header 1': '헤더1'}


### 결과 분석
- 각 텍스트 청크는 해당하는 모든 상위 헤더 정보를 메타데이터로 포함
- 계층 구조가 메타데이터에 보존되어 문맥 정보 유지
- 헤더가 없는 텍스트도 가장 가까운 상위 헤더의 메타데이터를 상속

## 2. 웹 URL에서 HTML 로드 및 다른 Splitter와 파이프라인 구성

### 파이프라인 구성 방법
1. HTMLHeaderTextSplitter로 구조적 분할
2. RecursiveCharacterTextSplitter로 크기 기반 재분할
3. 메타데이터 보존하면서 적절한 크기의 청크 생성

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 실제 웹 페이지 URL
url = "https://plato.stanford.edu/entries/goedel/"

# 더 많은 헤더 레벨 처리
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

# Step 1: HTML 구조 기반 분할
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text_from_url(url)

# Step 2: 크기 기반 재분할 설정
chunk_size = 500  # 각 청크의 최대 크기
chunk_overlap = 30  # 청크 간 중복 문자 수

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap
)

# Step 3: 파이프라인 실행
splits = text_splitter.split_documents(html_header_splits)

# 결과 샘플 출력 (80-85번째 청크)
for header in splits[80:85]:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

We see that Gödel first tried to reduce the consistency problem for analysis to that of arithmetic. This seemed to require a truth definition for arithmetic, which in turn led to paradoxes, such as the Liar paradox ('This sentence is false') and Berry's paradox ('The least number not defined by an expression consisting of just fourteen English words'). Gödel then noticed that such paradoxes would not necessarily arise if truth were replaced by provability. But this means that arithmetic truth
{'Header 1': 'Kurt Gödel', 'Header 2': '2. Gödel's Mathematical Work', 'Header 3': '2.2 The Incompleteness Theorems', 'Header 4': '2.2.1 The First Incompleteness Theorem'}
means that arithmetic truth and arithmetic provability are not co-extensive — whence the First Incompleteness Theorem.
{'Header 1': 'Kurt Gödel', 'Header 2': '2. Gödel's Mathematical Work', 'Header 3': '2.2 The Incompleteness Theorems', 'Header 4': '2.2.1 The First Incompleteness Theorem'}
This account of Gödel's discovery was t

### 파이프라인의 장점
- **구조 보존**: HTML 헤더 계층 구조가 메타데이터로 보존
- **크기 최적화**: RecursiveCharacterTextSplitter로 적절한 크기로 재분할
- **문맥 유지**: 청크 오버랩으로 문맥 연속성 확보

## 3. HTMLHeaderTextSplitter의 한계

### 주요 한계점
1. **HTML 구조 의존성**: 헤더가 표준적인 계층 구조를 따르지 않는 경우 문제 발생
2. **복잡한 레이아웃**: 헤더와 관련 텍스트가 별개의 하위 트리에 있는 경우 연결 실패
3. **동적 콘텐츠**: JavaScript로 생성되는 동적 콘텐츠는 처리 불가

### 실제 예시: CNN 뉴스 기사

In [3]:
# 복잡한 HTML 구조를 가진 실제 뉴스 사이트
url = "https://www.cnn.com/2023/09/25/weather/el-nino-winter-us-climate/index.html"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text_from_url(url)

# 처음 3개 결과만 출력
for header in html_header_splits[:3]:
    print(f"{header.page_content[:100]}")
    print(f"{header.metadata}", end="\n=====================\n")

CNN values your feedback  
1. How relevant is this ad to you?  
2. Did you encounter any technical i
{}
No two El Niño winters are the same, but many have temperature and precipitation trends in common.  
{'Header 2': 'What could this winter look like?'}
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad
{}


### 한계 분석
- **h1 메타데이터 누락**: 메인 헤드라인이 별도의 DOM 트리에 있어 메타데이터에 포함되지 않음
- **광고 콘텐츠 포함**: 관련 없는 광고 피드백 텍스트가 포함됨
- **h2는 정상 처리**: 표준적인 구조의 h2 헤더는 정상적으로 처리됨

## 4. 모범 사례 및 팁

### 사용 시 권장사항
1. **HTML 구조 사전 확인**: 대상 웹사이트의 HTML 구조를 먼저 분석
2. **적절한 헤더 선택**: 실제 콘텐츠 구조에 맞는 헤더 태그 선택
3. **파이프라인 활용**: 다른 splitter와 조합하여 최적의 청크 생성
4. **메타데이터 검증**: 분할 후 메타데이터가 올바르게 할당되었는지 확인

### 대안 고려
- 복잡한 HTML 구조: BeautifulSoup을 사용한 커스텀 파서 구현
- 동적 콘텐츠: Selenium 등을 사용한 렌더링 후 처리
- 비표준 구조: CSS 선택자 기반 커스텀 분할기 개발

## 요약

HTMLHeaderTextSplitter는 구조화된 HTML 문서를 의미론적으로 분할하는 강력한 도구입니다.

**장점**:
- 문서의 계층 구조를 메타데이터로 보존
- 다른 텍스트 분할기와 쉽게 결합 가능
- 웹 URL 직접 처리 지원

**한계**:
- 비표준적인 HTML 구조 처리의 어려움
- 동적 콘텐츠 처리 불가
- 복잡한 레이아웃에서의 제한사항

적절한 사용 사례를 선택하고 필요시 다른 도구와 결합하여 사용하면 효과적인 문서 처리가 가능합니다.